In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func#, text, inspect #added text and inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
stations = Base.classes.station
measurements = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
last_date = session.query(measurements.date).order_by(measurements.date.desc()).first()#[0]
print(last_date)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
last_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(last_year)

In [ ]:
# Perform a query to retrieve the date and precipitation scores
last_yr_prcp = session.query(measurements.date, measurements.prcp).\
    filter(measurements.date >= last_year).\
    filter(measurements.prcp != None).\
    order_by(measurements.date).all()
# added filter to remove null precipitation measurements

print(last_yr_prcp)

In [ ]:
# Save the query results as a Pandas DataFrame. Explicitly set the column names
prcp_df = pd.DataFrame(last_yr_prcp, columns=['Date', 'Precipitation'])
prcp_df

In [ ]:
# Sort the dataframe by date
#prcp_df.set_index('Date', inplace=True)
prcp_df = prcp_df.sort_values('Date')
prcp_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot(
    figsize=(8, 6), x='Date', #y='Precipitation',
    xlabel='Date', ylabel='Inches', #xlim=[last_year, ]
)
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
prcp_summary = prcp_df.describe()
prcp_summary

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
num_stations = session.query(stations).count()
num_stations

In [ ]:
station_list = session.query(stations.station).all()
print(station_list)

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_activity = session.query(measurements.station, func.count(measurements.station)).\
    group_by(measurements.station).\
    order_by(func.count(measurements.date).desc()).all()
station_activity


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active = station_activity[0][0] # get id of most active station

most_active_stats = session.query(func.min(measurements.tobs), func.max(measurements.tobs), func.avg(measurements.tobs)).\
    filter(measurements.station == most_active).all()
most_active_stats

#most_active_low = session.query(func.min(measurements.tobs)).\
#    filter(measurements.station == most_active).all()
#most_active_high = session.query(func.max(measurements.tobs)).\
#    filter(measurements.station == most_active).all()
#most_active_avg = session.query(func.avg(measurements.tobs)).\
#    filter(measurements.station == most_active).all()


In [ ]:
last_yr_tobs = session.query(measurements.date, measurements.tobs).\
    filter(measurements.date >= last_year).\
    filter(measurements.tobs != None).\
    filter(measurements.station == most_active).\
    order_by(measurements.date).all()
last_yr_tobs

In [ ]:
last_yr_tobs_df = pd.DataFrame(last_yr_tobs, columns=['dates', 'tobs'])
last_yr_tobs_df.set_index('dates', inplace=True)
last_yr_tobs_df

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
last_yr_tobs_df.plot(kind='hist',
                     column='tobs',
                     xlabel='Temperature',
                     bins=12
                     )

# Close Session

In [ ]:
# Close Session
session.close()